# Interferogram Handler
This ipynb contains code useful for viewing, finding, deleting, and renaming interferograms produced by OPUS from EM27 instruments. Often useful when ifgs are incorrectly labeled, we need information about datetimes within the metadata, etc. 

In [1]:
import datetime
import os

# Finding, Deleting, Renaming IFGs

In [ ]:
# Setting up path names -- you will need to adapt this to your system
ifg_data_folder = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs' #where the ifg date folders are
date = '20230516' #the date of interest -- I think this is the only one with bad ifgs for the TCCON side by side
date_folder = os.path.join(ifg_data_folder,date)

In [ ]:
#Find which ifgs are good and which are empty
good_data_ifgs = [] #ifgs with stuff in them
no_data_ifgs = [] #empty ifgs

for file in sorted(os.listdir(date_folder)): #loop through the data folder
    fsize_kb = os.stat(os.path.join(date_folder,file)).st_size/1024 #get the size of the file
    if fsize_kb>2: #if the filesize is bigger than 2kb
        good_data_ifgs.append(file) #add it to the good data list
    else:
        no_data_ifgs.append(file) #if not, add it to the no data list
print(f'IFGS with good data: {good_data_ifgs}')
print(f'IFGS with no data: {no_data_ifgs}')

In [ ]:
# BEFORE YOU RUN THIS CELL
# Make sure you have your data backed up as it will delete IFGS identified as having no data. 
# You will only have to run this once, then the files will be gone so you obviously cant "delete them again"
for file in no_data_ifgs:
    os.remove(os.path.join(date_folder,file))

In [ ]:
# Now we should have a folder with a gap in IFGS -- a set of good ifgs before xxx.3600 and a set of good
# ifgs after xxx.3600. We want to rename them so that we have a continuous set of data starting at xxx.0001

before_3600_ids = [] #all of the ifgs with an id before xxx.3600
after_3600_ids = [] #all of the ifgs with an id after xxx.3600
for file in sorted(os.listdir(date_folder)): #loop through the data folder
    ifg_id =int(file.split('.')[-1]) #get the ifg id by stripping off everything after the last "."
    if ifg_id < 3601: #if it's before 3601
        before_3600_ids.append(ifg_id) #add it to the before list
    else:
        after_3600_ids.append(ifg_id) #otherwise add it to the after list
last_before_3600_id = max(before_3600_ids) #this is the id of the last good ifg before 3600

In [ ]:
# AGAIN: you should only run this cell once and it will rename files in your directory, so make sure you know
# where you're working

new_id = last_before_3600_id #initialize the first "new_id" as the last good ifg before 3600
for old_id in after_3600_ids: #loop through all of the after 3600 ifgs, and rename them to be sequential starting with the last id before 3600
    new_id = new_id+1 #add one to the new_id, which will increment every time
    new_id_str = f'{new_id:04}' #make it a string of 4 digits with leading 0s 

    old_fullname = os.path.join(date_folder,f'ua{date}.ifg.{old_id}') #this is the old filename and path of the good ifg
    new_fullname = os.path.join(date_folder,f'ua{date}.ifg.{new_id_str}') #this is the new filename incremented one more than the previous
    os.rename(old_fullname,new_fullname) #this actually renames the file from the old to the new name

# Examining and resetting time in IFG files

View the datetime in the ifg binary file

In [1]:
# return the date and time strings, ensure that all printed output is what you expect
test_file = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs/20230516/ua20230516.ifg.0001'

print("File Path: " + test_file)
file = open(test_file, "rb")
s = file.read()
pos = s.find(b'DAT') # get the position of the string "DAT" in the ifg file
print("DAT position: " + str(pos)) #915428

# look for date string
# print(chr(s[pos]) + chr(s[pos+1]) + chr(s[pos+2]))
# print((s[pos:pos+3])) # prints "DAT"
date_start =  pos+8 # get index of where date string starts (8 characters from where the "DAT" string is found)
date_end = date_start + 10 # get index of where date string ends (10 characters from where the date string starts)
date_found = s[date_start:date_end] # get the complete date string using indices
print(date_found) # prints the date string in the file

# look for time string
pos = s.find(b'TIM') # get the position of the string "TIM" in the ifg file
time_start = pos+8 # get index of where time string starts (8 characters from where the "TIM" string is found)
time_end = time_start+20 # get index of where time string ends (20 characters from where the time string starts)
time_found = s[time_start:time_end] # get the complete time string using indices
print('TIM position: ' + str(pos))
print(time_found)

# # time_off = 42775 #in seconds
# # convert time_found to time object
# # add the timeoffset
# # convert back to string

file.close()

File Path: /uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs/20230516/ua20230516.ifg.0001
DAT position: 915428
b'16/05/2023'
TIM position: 915448
b'16:17:50.802 (GMT+0)'


Correct the datetime in the ifg binary file

In [ ]:
### Fix the time offset

## convert time string to time object 
tim_string = time_found.decode().split('.')[0]

## set date string
dat_string = date_found.decode() # use date found in file
# dat_string = "03/11/2022" # or if the date in the file is wrong, you can just indicate the correct date here

## convert date and time strings to datetime
test = datetime.datetime.strptime(dat_string + ' ' + tim_string, '%d/%m/%Y %H:%M:%S')
print("Original time: " + str(test))

## input time offset in seconds
time_off = datetime.timedelta(seconds = 61651)

## apply the time offset to get the correct time
new_time = test + time_off
print("Corrected time: " + str(new_time))

## create new byte object with the correct time string
new_time_str = str(new_time.time()) + '.704 (GMT+0)' # convert correct time to string, in the same format as in the ifg file
## note: I manually added the .704 to the end of the time so that the length of the time string would be the same length as 
##      what is in the ifg file. Without this it will not work, since it needs to replace text with a string with the same number
##      of characters. The times in the ifg files must be in some sort of decimal format, but I couldn't figure it out. I 
##      just accepted that my times may be off by ~1 second, but it's okay because I average it into 5 min anyway. 
new_time_byt = bytes(new_time_str, 'utf-8') # convert string to byte object
# print(new_time_byt)

In [ ]:
## replace the time_found with the corrected time
file = open(test_file, "rb")
s = file.read() 
s2 = s.replace(time_found, new_time_byt) # create a copy of the original file, but replace the times with the correct time
s2 = s2.replace(date_found, new_dat_byt) # replace the date with the correct date
file.close()

In [ ]:
## BE CAREFUL - THIS WILL OVERWRITE YOUR FILE!!!
## overwrite the file with the corrected time 
with open(test_file+'_test', 'wb') as output_file: # wb indicates we can write to the file
    output_file.write(s2)